<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/NN_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j=complex(0,1)
def fading_channel (Nt,Nr):
  h_real=np.random.normal(0, 0.5, size=(Nr, Nt))
  h_img=np.random.normal(0, 0.5, size=(Nr, Nt))
  H=np.array(h_real + h_img*j, dtype=complex)
  return H

In [3]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4,4))
Constellation_set[0]= np.array([1,0,0,0])
Constellation_set[1]= np.array([0,1,0,0])
Constellation_set[2]= np.array([0,0,1,0])
Constellation_set[3]= np.array([0,0,0,1])

In [4]:
H=fading_channel (4,2)

In [5]:
'''Complex noise'''
def noise (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  n_img=np.random.normal(0, N0, size=Nr)
  noise=np.array(n_real + n_img*j, dtype=complex)
  return noise

In [6]:
'''Creating pilot signals'''
def training (Label):
  i= 0
  y= np.zeros(shape= (4,2),dtype = "complex_")
  for i in range(Label):
    y[i]=np.dot(H, Constellation_set[i], out=None)+noise (0.1,2)
    i=i+1
  y_real=y.real
  y_imag=y.imag
  y1= np.concatenate((y_real,y_imag), axis=1)
  y2= pd.DataFrame(y1, columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
  y2['Label']=np.arange(0,4)
  return y2

In [7]:
'''Creating test signals'''
def test_ (size):
  y3=pd.DataFrame()
  for i in range(size):
    y= np.zeros(shape=(1,2),dtype = "complex_")
    rand=random.randrange(0, 4, 1)
    y=np.dot(H, Constellation_set[rand], out=None)+noise (0.1,2)
    y_real=y.real
    y_imag=y.imag
    y1= np.concatenate((y_real,y_imag), axis=None)
    y2=pd.DataFrame(y1.reshape((1,4)),columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
    if rand == 0 :
      y2['Label']= 0
    elif rand == 1 :
      y2['Label']= 1
    elif rand == 2 :
      y2['Label']= 2
    elif rand == 3 :
      y2['Label']= 3
    y3=pd.concat([y3,y2])
  return y3

In [8]:
test=test_ (2000)

In [9]:
n1=test['Label']
m1_NN=test[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
n1_NN=test['Label']
n1_NN=to_categorical(n1, num_classes=4)

In [10]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  return n_real

In [11]:
'''Data augmentation'''
q1=training (4)
l=pd.DataFrame()
for i in range(500):
  qs=q1[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]+noise_real(0.01,4)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1

In [12]:
xy_NN=pd.concat([l,q1])

In [13]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.3, random_state=101)

In [15]:
pip install keras-tuner --upgrade

     |████████████████████████████████| 98 kB 2.7 MB/s 


In [17]:

import keras_tuner as kt

In [18]:
def build_model(hp):
    model = keras.Sequential()
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 4)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=8, max_value=512, step=8),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(4, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(kt.HyperParameters())

In [19]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=5,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 8, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [21]:
tuner.search(X_train_NN, y_train_NN, epochs=3, validation_data=(m1_NN,n1_NN))

Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.9707000136375428

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 45s
INFO:tensorflow:Oracle triggered exit


In [22]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 4))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 464)               2320      
                                                                 
 dense_1 (Dense)             (None, 472)               219480    
                                                                 
 dense_2 (Dense)             (None, 216)               102168    
                                                                 
 dropout (Dropout)           (None, 216)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 868       
                                                                 
Total params: 324,836
Trainable params: 324,836
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tuner.results_summary()

In [24]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])

In [28]:
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(464, activation='relu'))
model.add(Dense(472, activation='relu'))
model.add(Dense(216, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(4, activation='softmax'))

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
NN=model.fit(X_train_NN, y_train_NN, batch_size=1, epochs=3, validation_data=(X_test_NN,y_test_NN))


Epoch 1/3
1402/1402 [==============================] - 10s 6ms/step - loss: 0.0210 - accuracy: 0.9964 - val_loss: 1.0366e-06 - val_accuracy: 1.0000
Epoch 2/3
1402/1402 [==============================] - 6s 4ms/step - loss: 3.5907e-06 - accuracy: 1.0000 - val_loss: 1.4238e-07 - val_accuracy: 1.0000
Epoch 3/3
1402/1402 [==============================] - 6s 4ms/step - loss: 1.0367e-06 - accuracy: 1.0000 - val_loss: 1.3267e-08 - val_accuracy: 1.0000


In [31]:
NN_Accuracy=model.evaluate(m1_NN, n1_NN)


63/63 [==============================] - 1s 5ms/step - loss: 6.1329e-04 - accuracy: 1.0000
